In [1]:
# Projection implementation of stochastic growth model adapted from Fabrice Collard's Matlab code, http://fabcol.free.fr/
# tested in Julia 1.5.0
# this code is part of chapter 5, "Advanced Numerical Techniques" from the book: "Introduction to Quantitative Macroeconomics using Julia"
# Academic Press - Elsevier
# for comments, email at: petre(dot)caraiani(at)gmail(dot)com

#packages
#Pkg.add("NLsolve");
using NLsolve;

using LinearAlgebra;

using Plots;

# The Optimal Growth Model
# Collocation method (Markov Chain case)                 

global kmin, ksup, XX, kt;
global nstate, nbk, ncoef, XX, XT, PI;

# Parameters                        
nbk   = 4;   # Degree of polynomials (capital)
nodes = nbk + 1;   # Nodes
nstate= 2; 
ncoef = nbk +1 ; # of coefficients
                                                           
# Structural Parameters                    
delta  = 0.1;
beta   = 0.95;
alpha  = 0.3;
sigma  = 1.5;
ysk    =(1-beta*(1-delta))/(alpha*beta);
ksy    = 1/ysk;
ys     = ksy^(alpha/(1-alpha));
ks     = ys^(1/alpha);
is     = delta*ks;
cs     = ys-is;
ab     = 0;


In [2]:
function hernodes(nstate)
TOL    = sqrt(2.2204e-16);
MAXIT  = 30;
PIM4   = pi^(-1/4);

n  = nstate;
m  = (n+1)/2;
m  =  convert(Int64, floor(m));
x  = zeros(n,1);
w  = zeros(n,1);

z  = 0;
z1 = 0;
pp = 0;
for i= 1:m

# Initialize the first four roots
if i == 1; 
   z= sqrt(2*n+1) - 1.85575 * (2*n+1)^(-1/6);
elseif i== 2;
   z= z - 1.14 * (n^.426)/z;
elseif i== 3;
   z= 1.86 * z - .86 * x[1];
elseif i== 4;
   z= 1.91 * z - .91 * x[2];
else;
   z= 2 * z - x[i-2];
end;

for its= 1:MAXIT
    p1 = PIM4;
    p2 = 0;
 for j  = 1:n
    p3 = p2;
    p2 = p1;
    p1 = z*sqrt(2/j)*p2 - sqrt((j-1)/j)*p3;
 end;

    pp = p2 * sqrt(2*n);
    z1 = z;
    z  = z1 - p1/pp;
if abs(z-z1) < TOL; break; end;
end;

x[i]    = z;
x[n+1-i]= -z;
w[i]= 2/(pp*pp);
w[n+1-i]= w[i];
end;       
        
return x,w
    end;

In [3]:
function transprob(ym,wm,m,r,s)
# this function computes the transition matrix
# Variables:
# ym is the vector of quadrature points
# wm is the vector of weights
# m is the mean of the process
# r is the rho of the process
# s is the conditional std.dev. of the process
    
n,n0=size(ym) ;        # get the number of quadrature points n
xx =  ones(n,1);       
xx=round.(Int64,vec(xx)); #make it integer: indices value in Julia must be integer
x=ym[:,xx] ;             # get x, nxn matrix, whose ji element is consumption
                         # growth in state j - so x is the value of
                         # consumption growth at time t, if the state is
                         # j at time t and i at time t+1 (notice it's
                         # constant across i)

y=x' ;                   # also an nxn matrix whose ji element is
                         # consumption growth in state i - so y is the
                         # value of consumption growth at time t+1, if
                         # the state is j at time t and i at time t+1
                         # (notice it's constant across j)

w=wm[:,xx]' ;     # converts the weight vector to a nxn matrix
                         # whose ji element is w(i) for all j
                          
f=(y.-m*(1-r)-x*r) ;
c1=exp.(-(f.*f)./(2.0*s*s))./(sqrt(2.0*pi)*s) ;


f=(y.-m*(1-r).-m*r) ;
c2=exp.(-(f.*f)./(2.0*s*s))./(sqrt(2.0*pi)*s) ;

p=(c1.*w)./c2 ;          # builds the transition matrix with elemets
                         # p(j,i)=f[y(i)|x(j)]w(i)/f[y(i)|mu]

sm=sum(p',dims=1)';
p=p./sm[:,xx];

return p;
end;

In [4]:
# Markov Chain technological process
rho  = 0.8;
se   = 0.2;
ma   = 0;
agrid,wmat=hernodes(nstate);
agrid=agrid*sqrt(2)*se; 
PI=transprob(agrid,wmat,0,rho,se)
at=agrid.+ma

2×1 Array{Float64,2}:
  0.2
 -0.2

In [5]:
#define functions rcheb transfo itransfo
function rcheb(nn);
mod=nn-floor(nn/2)*2;
n1=floor(nn/2);
if n1==1
k=(collect(1:n1))
    elseif n1>1
k=transpose(collect(1:n1));
    end;
r1=cos.((2*k.-1)*pi/(2*nn));
r1=[r1 -r1];
if mod==1;
  r1=[r1 0];
end;
r1=vec(r1);
rr=real(sort!(r1));
return rr
end;

function transfo(x,xmin,xmax);
  z=(2*(x.-xmin)./(xmax-xmin)).-1;
return z
end;

function itransfo(x,xmin,xmax);
z=0.5*(x.+1).*(xmax.-xmin).+xmin;
return z
end;

function cheb(xx,nn);
cc=real(cos.(kron(acos.(complex(xx)),nn)))      
return cc
    end;

In [6]:
# grid for the capital stock
kmin  = log(1.2);
ksup  = log(6);
rk    = rcheb(nodes);               #roots
kt    = exp.(itransfo(rk,kmin,ksup)) #grid
vnbk = transpose(collect(0:nbk));
XX    = cheb(rk,vnbk)

5×5 Array{Float64,2}:
 1.0  -0.951057      0.809017  -0.587785      0.309017
 1.0  -0.587785     -0.309017   0.951057     -0.809017
 1.0   6.12323e-17  -1.0       -1.83697e-16   1.0
 1.0   0.587785     -0.309017  -0.951057     -0.809017
 1.0   0.951057      0.809017   0.587785      0.309017

In [8]:
#initial conditions
a0=repeat([-0.2 0.65 0.04 0 0]',nstate,1);
a0=vec(a0);
param=[alpha beta delta sigma]';

In [9]:
# function makepoly

function makepoly(XA,XW);
nba   = size(XA,2);
nba1  = size(XA,1);
nbw   = size(XW,2);
nmax  = max(nba,nbw);
XX    = Float64[]  ;  
    
for i=1:nbw
   for j=1:nba         
         XX=[XX; kron(XW[:,i],XA[:,j])];
   end
end
return XX
    end;

In [10]:
f  = function(theta);
RHS=Float64[];
LHS=Float64[];
resid=Float64[]; 
ct = 0.0;    
c1 = 0.0;  
lt = length(theta);
lt = round.(Int64,lt/nstate);
theta = reshape(theta,lt,nstate);

for i   = 1:nstate
        
        ct   = exp.(XX*theta[:,i]);
        k1   = exp.(at[i])*kt.^alpha+(1-delta)*kt-ct;
        rk1  = transfo(log.(k1),kmin,ksup);
        vnbk = transpose(collect(0:nbk));    
        xk1  = cheb(complex(rk1),vnbk);
        aux  = 0;

        for j=1:nstate;
        c1   = exp.(xk1*theta[:,j]);
        aux  = aux.+PI[i,j]*beta*(alpha*exp.(at[j])*k1.^(alpha-1).+(1-delta)).*c1.^(-sigma);
        end;
        
        RHS  = [RHS;-sigma*log.(ct)];
        LHS  = [LHS;log.(aux)];
end;
resid     = LHS-RHS;
return vec(resid);
end;

In [11]:
#Pkg.add("NLsolve");
using NLsolve;
sol  = nlsolve(f,a0);
fsol = reshape(sol.zero,ncoef,nstate);
println("Display final rule:",fsol)


Display final rule:[0.16195515836563915 0.01630262920045456; 0.34349362216110896 0.3831777478024939; 0.009496909427925975 0.008433901793600694; 4.2768574038867555e-5 -7.841805118970527e-5; -1.0607581975470355e-5 -1.2227920204005528e-6]


In [12]:
#final
lt  = length(sol.zero);
nb  = 1000;
kt  = collect(kmin:(ksup-kmin)/(nb-1):ksup);
rk  = transfo(kt,kmin,ksup);
rk  = vec(rk);
XX  = cheb(rk,vnbk);
kt  = exp.(kt);
ct=[];k1=[];ii=[];
for i=1:nstate
    if i==1
    ct  = exp.(XX*fsol[:,i]);
    k1  = exp.(at[i])*kt.^alpha+(1-delta)*kt-ct[:,i];
    ii  = exp.(at[i])*kt.^alpha-ct[:,i];
    else
    ct  = hcat(ct, exp.(XX*fsol[:,i]));
    k1  = hcat(k1, exp.(at[i])*kt.^alpha+(1-delta)*kt-ct[:,i]);
    ii  = hcat(ii,exp.(at[i])*kt.^alpha-ct[:,i]);
    end;
end;   

In [13]:
#using Plots
plotly() 
plot(kt,ct, linewidth=1,title="Consumption vs Capital Stock", label="Consumption")


┌ Info: For saving to png with the Plotly backend ORCA has to be installed.
└ @ Plots C:\Users\PETRE\.julia\packages\Plots\5srrj\src\backends.jl:371


<!DOCTYPE html>
 
 
 Plots.jl